## A basic strain design tutorial using MEWpy

Let's start by loading our model...

In [ ]:
from reframed import load_cbmodel

model = load_cbmodel("models/e_coli_core.xml.gz", flavor="bigg")

We will define two optimization objectives:

* Maximize the flux of our target reaction (succinate production)
* Maximize the Biomass-Product Coupled Yield (finds an optimal trade-off between growth and production)

In [ ]:
from mewpy.optimization.evaluation import BPCY, TargetFlux

objs = [
    TargetFlux("R_EX_succ_e"), 
    BPCY(model.biomass_reaction, "R_EX_succ_e")
]

Now we define our optimization problem:
* Type of modifications we are searching for (reaction knockouts)
* Environmental conditions (anaerobic growth)

In [ ]:
from mewpy.problems import RKOProblem

anaerobic = {'R_EX_o2_e': (0, 0)}
problem = RKOProblem(model, fevaluation=objs, envcond=anaerobic)

And now we can run the optimization an evolutionary algorithm (EA)

In [ ]:
from mewpy.optimization import EA

solutions = EA(problem, max_generations=100).run()

By default, MEWpy calculates a population of the best 100 solutions.

To make our life easier, let's convert the result to a Pandas DataFrame:

In [ ]:
import pandas as pd

get_list = lambda x: [r_id[2:] for r_id in x.values]
table = [[get_list(x), len(get_list(x)), x.fitness[0], x.fitness[1]] for x in solutions]
df = pd.DataFrame(table, columns=["reactions", "knockouts", "target", "BPCY"])

We can now sort the results, for instance by the total number of required knockouts:

In [ ]:
df.sort_values("knockouts")

We can also look at the trade-off between our two objectives (the so-called Pareto front)

In [ ]:
df.plot.scatter("BPCY", "target")